##### Copyright 2021 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Learning to Cluster using Neural Clustering Process (NCP) 

Clustering has traditionally been posed as an unsupervised learning problem. However, unsupervised clustering methods often rely on strong assumptions about data geometry, and the clusters produced might not agree with human labels. Recently, supervised formulations of clustering have gained increasing interest, with novel objective functions and neural network architectures that directly learn complex clustering objectives from labeled data. In this tutorial, we will demonstrate a supervised neural clustering method, [Neural Clustering Process (NCP)](https://arxiv.org/pdf/1901.00409.pdf), which can be trained end-to-end to perform amortized probabilistic clustering without assuming a fixed or maximum number of clusters.


The high-level steps that we will follow in this colab are:
1. Create synthetic data for clustering using a [Mixture of Gaussians (MOG)](https://en.wikipedia.org/wiki/Mixture_model#Gaussian_mixture_model) generative model.
2. Define a NCP model architecture based on multi-layer perceptrons (MLP).
3. Train the NCP model on synthetic MOG data.
4. Perform probabilistic clustering using the trained NCP model.

## Setup

Install the `neural_clustering` module by running the code cell below.

In [ ]:
!pip install 'git+https://github.com/tensorflow/neural-structured-learning.git#egg=neural_clustering&subdirectory=research/neural_clustering'

In [ ]:
import time

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn import metrics

from neural_clustering import models
from neural_clustering import data_generators

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
COLORS = np.array(sns.color_palette() + sns.color_palette("pastel"))

## Generating and visualizing synthetic data

The synthetic data will be generated in two steps. 

- First, we will produce random cluster partitions using a [Chinese Restaurant Process (CRP)](https://en.wikipedia.org/wiki/Chinese_restaurant_process) generative model. 

- Given the cluster partitions, synthetic data points in each cluster will then be generated using [Mixture of Gaussians (MOG)](https://en.wikipedia.org/wiki/Mixture_model#Gaussian_mixture_model). 

The code below instantiates a MOG data generator that produces 2 dimensional data so that we can visualize them easily. We also specify the discount parameter of CRP and the prior parameters of the mixture distribution.

In [ ]:
crp_generator = data_generators.CRPGenerator(alpha=0.8)
mog_generator = data_generators.MOGGenerator(
    partition_generator=crp_generator,
    x_dim=2,
    prior_sigma=10,
    sigma_min=0.5,
    sigma_max=2)

Let's creates some synthetic data and plot them for visualization. 

In this example, we generate a mini-batch of four MOG samples. The parameter `N` specifies the number of data points in each sample. The plot shows that each sample contains a variable number of clusters and a different distribution of data points. 

In [ ]:
np.random.seed(8)
N = 1000  #@param {"type": "number"}
batch_size = 4

mog_data, mog_labels = mog_generator.generate_batch(N, batch_size=batch_size)
print("data shape:", mog_data.shape)
print("labels shape:", mog_labels.shape)

fig, axes = plt.subplots(1, batch_size, figsize=(batch_size * 4.5, 4))
for i, ax in enumerate(axes):
  ax.scatter(mog_data[i, :, 0], mog_data[i, :, 1], s=0.3, c=COLORS[mog_labels[i]])
  ax.set_title(f'Sample {i}')
plt.show()

## Neural Clustering Process (NCP) model

An end-to-end NCP clustering model contains the following two components:

- **An encoder network** that transforms the input data into a fixed-dimensional vector. The type of encoder depends on the structure of the input data. For example, a convolutional neural network can be used for image data, and a graph neural network for graph data, etc. Here, we use a simple multi-layer perceptron for our MOG data.

- **A NCP network** that takes as input the encoded vectors and learns to cluster these data points. The NCP algorithm assigns cluster labels to N data points sequentially by N forward passes throught the network. Our implementation decomposes the algorithm into two parts. The [`NCPBase`](https://github.com/tensorflow/neural-structured-learning/blob/master/research/neural_clustering/models/ncp_base.py) model class handles the forward pass for one iteration of NCP, and the [`NCPWrapper`](https://github.com/tensorflow/neural-structured-learning/blob/master/research/neural_clustering/models/ncp_wrapper.py) class wraps an [`NCPBase`](https://github.com/tensorflow/neural-structured-learning/blob/master/research/neural_clustering/models/ncp_base.py) model to perform the full sequential clustering algorithm. This is similar to the idea of [wrapping RNN cells into RNN layers](https://www.tensorflow.org/guide/keras/rnn). Both `NCPBase` and `NCPWrapper` are subclasses of `tf.keras.Model`.

These two networks are then trained jointly through backpropagation.

The [`NCPBase`](https://github.com/tensorflow/neural-structured-learning/blob/master/research/neural_clustering/models/ncp_base.py) model class itself does not define any neural network layer. Rather, it implements the NCP algorithm and expects the users to provide the neural network structures as its building-block layers. This provides maximal flexibility on the model architecture. In the code below, we use the [`NCPWithMLP`](https://github.com/tensorflow/neural-structured-learning/blob/master/research/neural_clustering/models/ncp_models.py) model, a subclass of [`NCPBase`](https://github.com/tensorflow/neural-structured-learning/blob/master/research/neural_clustering/models/ncp_base.py) that defines the building-block layers of NCP as multi-layer perceptrons (MLP).

In [ ]:
# The default dimension size of hidden layers.
layer_dim = 128

# An encoder model for MOG data. Here we use a multi-layer perceptron (MLP).
mog_encoder = tf.keras.Sequential([
    tf.keras.layers.Dense(layer_dim//2, activation="relu", name="e1"),
    tf.keras.layers.Dense(layer_dim//2, activation="relu", name="e2"),
    tf.keras.layers.Dense(layer_dim, name="e3"),
])

# A NCP model with multi-layer perceptrons (MLP) as its building blocks.
ncp_model = models.NCPWithMLP(
    assigned_point_layer_out_dim=layer_dim,
    assigned_point_layer_hidden_dims=[layer_dim] * 2,
    unassigned_point_layer_out_dim=layer_dim,
    unassigned_point_layer_hidden_dims=[layer_dim] * 2,
    cluster_layer_out_dim=layer_dim,
    cluster_layer_hidden_dims=[layer_dim] * 2,
    logits_layer_hidden_dims=[layer_dim] * 2)

# The NCPWrapper class wraps the base model for the full sequential clustering
# algorithm. The sampler parameter is used at inference time for predicting the
# cluster labels. It's set to None during training because training labels are
# available.
ncp_wrapper_model = models.NCPWrapper(ncp_model=ncp_model, sampler=None)


## Training the NCP clustering model

In order to train a NCP model given input data and target cluster labels, we minimize the negative log likelihood loss using the `logits` returned from `ncp_wrapper_model`. In the `train_step` function below, we define a custom training loop that jointly updates the weights of the encoder network and NCP network by gradient descent.

In [ ]:
optimizer = tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.0001)

def train_step(inputs, targets):
  with tf.GradientTape() as tape:
    encoded = mog_encoder(inputs)
    logits, _ = ncp_wrapper_model(encoded, targets, training=True)
    loss = ncp_wrapper_model.loss_function(logits)

  variables = ncp_wrapper_model.trainable_variables + mog_encoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  
  # returns the loss averaged by the number of data points.
  return loss / inputs.shape[1]

Now let's train the clustering model. For the purpose of illustration, we use a small number of training steps.

In each mini-batch, we generate synthetic training data similar to the examples shown earlier. The total number of data points to cluster in each training step is randomly sampled from a range, so that the model learns to cluster data of different sizes. The synthetic training examples across the mini-batch may also contain different number clusters.

In [ ]:
training_steps = 500  #@param {"type": "number"}
n_points_min = 50  #@param {"type": "number"}
n_points_max = 200  #@param {"type": "number"}
batch_size = 32
losses = []

for i in range(training_steps):
  n_total_points = np.random.randint(n_points_min, n_points_max)
  batch_data, labels = mog_generator.generate_batch(n_total_points, batch_size=batch_size) 

  t_start = time.time()
  loss = train_step(batch_data, labels)
  batch_time = time.time() - t_start

  losses.append(loss.numpy())
  if i % 10 == 0:
    print(f'Batch {i}, Number of data points: {n_total_points}, Loss: {np.mean(losses[-10:]):.3g}, Time/Batch: {batch_time:.2f}s')

In [ ]:
plt.plot(np.convolve(losses, np.ones(5)/5, mode='valid'))
plt.ylabel("Loss")
plt.xlabel("Batch")

## Probabilistic clustering using trained NCP models

The trained NCP model can be used to cluster new data points. Remember that NCP assigns cluster labels to each data point sequentially. In each iteration, NCP computes the unnormalized log probability (logits) of whether the new point should join one of the existing clusters or create a new one. This probability is then used for cluster assignment.

Here, we demonstrate two methods for clustering assignment. The **greedy sampler** below always assigns a new point to the cluster with the highest probability, whereas the **random categorical sampler** will randomly sample a cluster label based on the categorical distribution defined by the logits. 

In the examples below, we will create a `ncp_sampler` by wrapping the trained `ncp_model` together with a sampler class, either the `GreedySampler` or the `CategoricalSampler`.

We will generate some test data, and ask the trained NCP model to cluster them. The ground truth and inferred cluster labels will be visualized in plots. The joint probability of the final clustering configuration are computed using the logits from all cluster assignments, and shown in the plots below (e.g. `p=0.035`). We also evaluate the clustering quality using [adjusted mutual information (AMI)](https://en.wikipedia.org/wiki/Adjusted_mutual_information), which measures the agreement between the predicted cluster labels and ground truth. The AMI score ranges from 0 to 1, and a higher score indicates higher clustering quality.

In [ ]:
# Helper function for plotting the clustering results.
def plot_clusters(data, labels, predictions, logits):
  batch_size = data.shape[0]
  joint_probs = np.exp(tf.reduce_sum(logits, axis=1))
  ami_scores = [metrics.adjusted_mutual_info_score(labels[i], predictions[i]) for i in range(batch_size)]

  _, axes = plt.subplots(2, batch_size, figsize=(4 * batch_size, 8))
  for i in range(batch_size):
    axes[0, i].scatter(data[i, :, 0], data[i, :, 1], s=3, c=COLORS[labels[i]])
    axes[0, i].set_title(f'Test Example {i}\nGround Truth')
    axes[0, i].get_xaxis().set_visible(False)
    axes[0, i].get_yaxis().set_visible(False)
    axes[1, i].scatter(data[i, :, 0], data[i, :, 1], s=3, c=COLORS[predictions[i].numpy()])
    axes[1, i].set_title(f'Inferred\np = {joint_probs[i]:.4f}, AMI = {ami_scores[i]:.3f}')
    axes[1, i].get_xaxis().set_visible(False)
    axes[1, i].get_yaxis().set_visible(False)

### Test data

In [ ]:
seed = 5  #@param {"type": "number"}
np.random.seed(seed)
n_points = 200  #@param {"type": "number"}
batch_size = 4
test_data, test_labels = mog_generator.generate_batch(n_points, batch_size=batch_size) 

### Clustering using NCP + greedy sampler


In [ ]:
ncp_greedy_sampler = models.NCPWrapper(ncp_model=ncp_model, sampler=models.GreedySampler())

test_encoded = mog_encoder(test_data)
logits, test_pred = ncp_greedy_sampler(test_encoded, training=False)

plot_clusters(test_data, test_labels, test_pred, logits)

### Clustering using NCP + random categorical sampler

In [ ]:
ncp_categorical_sampler = models.NCPWrapper(ncp_model=ncp_model, sampler=models.CategoricalSampler())

test_encoded = mog_encoder(test_data)
logits, test_pred = ncp_categorical_sampler(test_encoded, training=False)

plot_clusters(test_data, test_labels, test_pred, logits)

### Capturing clustering uncertainty by parallelized sampling

In a single run, the greedy sampler usually produces clustering configurations with higher probabilities and AMI scores than the random categorical sampler. However, the random categorical sampler can be used to sample many sets of cluster labels in parallel, and by choosing the output with the highest probability, we may get higher quality clusters than by using the greedy sampler. 

Importantly, this random sampling capability of NCP can be used to capture the uncertainty of cluster assignment by generating multiple plausible clustering configurations.

In the example below, we create one test example, and ask NCP to sample multiple sets of cluster labels. The top few clustering results with high probability are shown in the plots.

In [ ]:
# Test data.
seed = 1  #@param {"type": "number"}
np.random.seed(1)
n_points = 200  #@param {"type": "number"}
test_data, test_labels = mog_generator.generate_batch(n_points, batch_size=1) 

test_encoded = mog_encoder(test_data)

# The encoded tensor is replicated to perform parallelized sampling of cluster labels.
n_parallel_samples = 100  #@param {"type": "number"}
test_encoded_replicated = tf.repeat(test_encoded, repeats=n_parallel_samples, axis=0)

logits, test_pred = ncp_categorical_sampler(test_encoded_replicated, training=False)
joint_probs = np.exp(tf.reduce_sum(logits, axis=1))

# Sorting the unique predictions by probability.
# (np.unique returns sorted unique elements.)
joint_probs, unique_index = np.unique(joint_probs, return_index=True)
joint_probs, unique_index = joint_probs[::-1], unique_index[::-1]
test_pred = tf.gather(test_pred, unique_index)

# Only plotting the top few predictions.
top_n = min(4, len(joint_probs))

ami_scores = [metrics.adjusted_mutual_info_score(test_labels[0], test_pred[i]) for i in range(top_n)]

_, axes = plt.subplots(1, top_n + 1, figsize=(4 * (top_n + 1), 4))
axes[0].scatter(test_data[0, :, 0], test_data[0, :, 1], s=3, c=COLORS[test_labels[0]])
axes[0].set_title(f'Ground Truth')

for i in range(top_n):
  axes[i+1].scatter(test_data[0, :, 0], test_data[0, :, 1], s=3, c=COLORS[test_pred[i].numpy()])
  axes[i+1].set_title(f'Inferred #{i+1}\np = {joint_probs[i]:.4f}, AMI = {ami_scores[i]:.3f}')

## Conclusion

In this colab, we have created a Neural Clustering Process (NCP) model, and trained the model to cluster synthetic data. We have seen that NCP can learn to perform probabilistic clustering on datasets of different sizes and containing different number of clusters.